### CREAMOS EL ARCHIVO SQL


In [1]:
with open("consulta.sql", "w") as file:
    file.write("SELECT * FROM Ejemplo;")

### Librerias y LLamar a Lector_deConsulta.py

In [4]:
!python Lector_deConsulta.py

Conexión exitosa
/home/roma/ENTREGAS/ENTREGAS/Lector_deConsulta.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
Datos guardados en '/home/roma/ENTREGAS/ENTREGAS/Carpeta_ejemplo/resultado.parquet'
Conexión cerrada.


### VERIFICAMOS QUE ESTE BIEN EL ARCHIVO CREADO

In [ ]:
# Leer config.json para conocer la carpeta de salida
import os
import json
import pandas as pd

with open("config.json", "r") as f:
    config_json = json.load(f)

# Ruta dinámica desde el notebook
notebook_dir = os.getcwd()
output_dir = os.path.join(notebook_dir, config_json["output_folder"])
output_file = os.path.join(output_dir, "resultado.parquet")

print(os.listdir(output_dir))
df = pd.read_parquet(output_file)
print(df)


['resultado.parquet']
   Casas  Empleado  Animales  Apartamente  Carros
0      1         2         3            4       5
1      2         3         4            5       6
2      3         4         5            6       7
3      8         9         0            1       2
4      3         4         5            6       7
5      8         9         0            1       2
6      3         4         5            6       7
7      8         9         0            1       2
8      3         4         5            6       7
9      8         9         0            1       2
